In [0]:
import quandl
import pandas as pd
import numpy as np
from google.colab import drive
import os
import time
import urllib
import json
import csv
import requests
import io
from pathlib import Path
import random

drive.mount('/content/gdrive')


quandl.ApiConfig.api_key = 'jsb-w4XT_q1wk2Ey_PT-'
#quandl.ApiConfig.api_version = '2015-04-09'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# googlepath exists for googledrive finding the files. 
# string if running on computer. Have companylist.csv in same dir. Will create
# a stock_data folder and download stocks data into that. 
googlepath = "gdrive/My Drive/Colab Notebooks/CS230_PROJECT/" 


In [0]:
#mydata = quandl.get("WIKI/VIX", rows=5)
def save(googlepath, stock_csv, output_dir, filename):
    try:
        #the output dir may not exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
    except Exception as ex:
        print('Could not create output dir')
        print(ex)
        return
    filepath = os.path.join(googlepath, output_dir, filename)
    try:
#         print(stock_csv)
        df = stock_csv
        #df = df.sort_values(by='timestamp')  
#         print(df)
        df.to_csv(filepath, index=False)
    except Exception as ex:
        print('Could not open file {} to write data'.format(filepath))
        print(ex)

def try_download(symbol):
    try:
        # Keep call frequency below threshold 
        time.sleep(10)          

        # Quandl Download
        data_EQ= quandl.get_table('SHARADAR/SEP', ticker=symbol) # Daily Equity Price
        df_EQ = pd.DataFrame(data=data_EQ)
        data_DM= quandl.get_table('SHARADAR/DAILY', ticker=symbol) # Daily Metrics
        df_DM = pd.DataFrame(data=data_DM)
        data_CO= quandl.get_table('SHARADAR/ACTIONS') # Corporate Actions
        df_CO = pd.DataFrame(data=data_CO)
        data_SF1= quandl.get_table('SHARADAR/SF1', ticker=symbol) # Core US Fundamentals
        df_SF1 = pd.DataFrame(data=data_SF1)


        # Print the Head of df_EQ y and shows data
        #df_EQ

        # Merge to build 'symbol_Basic' a dataset that merged DF_EQ and DF_DM
        symbol_Basic=pd.merge(df_EQ,df_DM,on='date',sort=True) 
        #symbol_Basic.columns

        symbol_Basic=symbol_Basic.drop(['closeunadj','lastupdated_x', 'ticker_y', 'lastupdated_y'], axis=1)
        #symbol_Basic

        # Generate a Stock Data File, Small Set of Features
        #time.sleep(10)
        #symbol_Basic.to_csv (r'gdrive/My Drive/Colab Notebooks/CS230_PROJECT/stock_data/ETSY_symbol_Basic.csv', index = True, header=True )

        # In the SF1 dataset just get the ART: trailing-twelve-months ( in 'dimension' column)
        df_SF1_ART=df_SF1.loc[df_SF1.dimension=="ART"]
        df_SF1_ART=df_SF1_ART.drop(['ticker','dimension', 'calendardate','reportperiod', 'lastupdated'], axis=1)

        # Change label of 'datekey' and 'date to do the Merge
        df_SF1_ART = df_SF1_ART.rename(columns={'datekey': 'date'})

        symbol_Premium=pd.merge(symbol_Basic, df_SF1_ART, on='date',how='outer',sort=True)

        # Data Augmentation to use Fundamental Data for 90 days after ART release
        symbol_Premium=symbol_Premium.fillna(method='ffill')
        
        # Drop Values does not have Fundamental Data
        symbol_Premium=symbol_Premium.dropna()
        
        # Generate a Stock Data File all Features
        #time.sleep(10)  
        #symbol_Premium.to_csv (r'gdrive/My Drive/Colab Notebooks/CS230_PROJECT/stock_data/ETSY_Premium.csv', index = True, header=True )

        return symbol_Premium, True
    except Exception as ex:
        print(ex)
        return None, None



#output dir as a csv 

def download_symbol(symbol, output_dir, retry_count=4):

    stock_csv, didPass = try_download(symbol)
    if didPass:
        save(googlepath, stock_csv, output_dir, '{}.csv'.format(symbol))
    else:
        print('Failed to download {}'.format(symbol))

df = pd.read_csv(f"{googlepath}companylist.csv")
for symbol in df.Symbol:
    my_file = Path(f"{googlepath}stock_data/{symbol}.csv")  # check if already downloaded
#     print(my_file.exists())
    if not my_file.exists():
        print('Downloading {}'.format(symbol))
        download_symbol(symbol, 'stock_data')
    else:
        print(f"Already downloaded {symbol}")




You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat
Failed to download GOOG
You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat
Failed to download DISCK
You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat
Failed to download NWS
You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat
Failed to download FOX
You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat
Failed to download UA
